In [1]:
#import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

In [2]:
model = Sequential()

In [3]:
print("np version:", np.__version__)
print("tensorflow version:", tf.__version__)
print("keras version:", tf.keras.__version__)

np version: 1.26.4
tensorflow version: 2.19.0
keras version: 3.9.0


In [4]:
model.add(Conv2D(64,(3,3) , input_shape=(128,128,3) ,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3) ,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=7,activation='softmax'))  

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [6]:
#image argument process    '
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8,1.2]
)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/train',
                                                target_size = (128, 128),
                                                batch_size = 32,
                                                class_mode = 'sparse')

test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'sparse')



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
model.fit(training_set,
                steps_per_epoch = 28709 ,
                epochs = 1,
                validation_data = test_set,
                validation_steps = 7178 )

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  898/28709 ━━━━━━━━━━━━━━━━━━━━ 3:53:34 504ms/step - accuracy: 0.2370 - loss: 1.8695

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


28709/28709 ━━━━━━━━━━━━━━━━━━━━ 626s 22ms/step - accuracy: 0.2486 - loss: 1.8094 - val_accuracy: 0.2719 - val_loss: 1.7480


In [9]:

test_image = image.load_img('surprise.jpg',target_size=(128,128))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

class_labels = ['angry', 'disgust', 'fear','happy','neutral','sad','surprise']

# Predict
result = model.predict(test_image)

print(result)
predicted_index = np.argmax(result)
predicted = class_labels[predicted_index]

# Interpret result
print(f"Predicted: {predicted}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[0.09680893 0.01336124 0.15767573 0.28740096 0.1489373  0.1354618
  0.16035406]]
Predicted: happy


In [11]:
model.save('model.h5')